<a href="https://colab.research.google.com/github/YairZen/CloudProject_Unicorn/blob/lior/HW2_UNICORN_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ PIP INSTALL (GLOBAL)

In [60]:
!pip install -q gradio pandas matplotlib
!pip install python-docx

✅ IMPORTS (GLOBAL)

In [61]:
import gradio as gr
from transformers import pipeline
import requests
import pandas as pd
import matplotlib.pyplot as plt
#######
# Report Generator with DOCX Export
from docx import Document
import tempfile
import os
from datetime import datetime
import pandas as pd
from docx.enum.text import WD_ALIGN_PARAGRAPH

✅ GLOBAL CONFIG / THEME / CSS (OPTIONAL)

In [62]:
APP_TITLE = "🌱 CloudGarden"
APP_SUBTITLE = "Smart Plant Disease Detection System"
BASE_URL = "https://server-cloud-v645.onrender.com/"

# --- Colors (hard-coded) ---
COLOR_TEMP = "#1f77b4"   # blue
COLOR_HUM  = "#ff7f0e"   # orange
COLOR_SOIL = "#2ca02c"   # green

STATUS_OK_COLOR = "#2ca02c"      # green
STATUS_WARN_COLOR = "#ffbf00"    # yellow
STATUS_BAD_COLOR = "#d62728"     # red

# Model from the tutorial
MODEL_NAME = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"

clf = pipeline(
    "image-classification",
    model=MODEL_NAME
)


CUSTOM_CSS = """
/* כאן אפשר לשים CSS גלובלי לאתר */
"""

/usr/local/lib/python3.12/dist-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:30: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(
Device set to use cpu


✅ TAB REGISTRY (MODULAR)

 מוסיפים חלונית חדשה רק ע"י:
 1) הוספת TAB1 Logic - כל הפונקציות עזר למינהם.
 2) הוספת Tab1 GUI - כל GRADIO
 * יש להוריד את "with gr.Blocks() as demo:" ו "demo.launch()"

 3) להוסיף את שם הפונקציית GUI לרשימת הTAB למטה.

✅ TAB 1 Logic -  🌱 Realtime Dashboard


In [63]:

# ---------- Core Data Fetch ----------
def load_iot_data(feed: str, limit: int) -> pd.DataFrame | None:
    resp = requests.get(
        f"{BASE_URL}/history",
        params={"feed": feed, "limit": limit},
        timeout=30
    )
    data = resp.json()
    if "data" not in data or not data["data"]:
        return None

    df = pd.DataFrame(data["data"])
    if "created_at" not in df.columns or "value" not in df.columns:
        return None

    df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["created_at", "value"]).sort_values("created_at")

    return None if df.empty else df


# ---------- Helpers ----------
def normalize(series: pd.Series) -> pd.Series:
    mn, mx = float(series.min()), float(series.max())
    if mx - mn == 0:
        return series * 0.0
    return (series - mn) / (mx - mn)


# ---------- Plant Status + Plots ----------
def plant_dashboard(limit: int):
    try:
        dfs = {
            "temperature": load_iot_data("temperature", limit),
            "humidity": load_iot_data("humidity", limit),
            "soil": load_iot_data("soil", limit),
        }

        missing = [k for k, v in dfs.items() if v is None]
        if missing:
            return "⚠️ Partial Data", f"Missing sensors or empty history: {', '.join(missing)}", None, None, None, None

        temp = float(dfs["temperature"]["value"].iloc[-1])
        hum = float(dfs["humidity"]["value"].iloc[-1])
        soil = float(dfs["soil"]["value"].iloc[-1])

        issues, warnings = [], []

        checks = [
            ("Temperature", temp, 18, 32, 1),
            ("Air humidity", hum, 35, 75, 3),
            ("Soil moisture", soil, 20, 60, 3),
        ]

        for name, value, low, high, margin in checks:
            if not (low <= value <= high):
                issues.append(f"{name} out of range ({value:.1f})")
            elif value <= low + margin or value >= high - margin:
                warnings.append(f"{name} near limit ({value:.1f})")

        if issues:
            status = "🔴 Plant Status: Not OK"
            details_main = " ; ".join(issues)

        elif warnings:
            status = "🟡 Plant Status: Warning"
            details_main = " ; ".join(warnings)

        else:
            status = "🟢 Plant Status: OK"
            details_main = "All sensors are within valid ranges"

        details = (
    f"{details_main}\n"
    f"Latest values:\n"
    f"temp={temp:.1f}\n"
    f"humidity={hum:.1f}\n"
    f"soil={soil:.1f}"

        )

        df_t, df_h, df_s = dfs["temperature"], dfs["humidity"], dfs["soil"]

        fig_t = plt.figure(figsize=(7, 3.2))
        plt.plot(df_t["created_at"], df_t["value"], marker="o", color=COLOR_TEMP)
        plt.title("Temperature History")
        plt.xlabel("Time")
        plt.ylabel("°C")
        plt.grid(True)

        fig_h = plt.figure(figsize=(7, 3.2))
        plt.plot(df_h["created_at"], df_h["value"], marker="o", color=COLOR_HUM)
        plt.title("Air Humidity History")
        plt.xlabel("Time")
        plt.ylabel("%")
        plt.grid(True)

        fig_s = plt.figure(figsize=(7, 3.2))
        plt.plot(df_s["created_at"], df_s["value"], marker="o", color=COLOR_SOIL)
        plt.title("Soil Moisture History")
        plt.xlabel("Time")
        plt.ylabel("%")
        plt.grid(True)

        fig_c = plt.figure(figsize=(10, 3.4))
        plt.plot(df_t["created_at"], normalize(df_t["value"]), marker="o", label="Temperature (norm)", color=COLOR_TEMP)
        plt.plot(df_h["created_at"], normalize(df_h["value"]), marker="o", label="Humidity (norm)", color=COLOR_HUM)
        plt.plot(df_s["created_at"], normalize(df_s["value"]), marker="o", label="Soil (norm)", color=COLOR_SOIL)
        plt.title("Combined Trend (Normalized)")
        plt.xlabel("Time")
        plt.ylabel("Normalized Value (0–1)")
        plt.grid(True)
        plt.legend()

        return status, details, fig_t, fig_h, fig_s, fig_c

    except Exception:
        return "❌ Error", "Failed to fetch data from server. Please try again.", None, None, None, None


✅ Tab 1 GUI - - 🌱 Realtime Dashboard


In [64]:
def build_realtime_dashboard_tab():
    gr.Markdown(
    "<h3 style='margin:0; font-size:22px;'>🌿 Overall Plant Status (Real-Time)</h3>"
)


    samples = gr.Slider(1, 200, value=20, step=1, label="Number of Samples (used for all graphs)")
    overall_btn = gr.Button("Update Plant Dashboard", variant="primary")



    overall_status = gr.Textbox(
        label="Overall Status",
        lines=1,
        placeholder="Click 'Update Plant Dashboard' to evaluate plant status"
    )
    overall_info = gr.Textbox(
        label="Status Details",
        lines=4,
        placeholder="Detailed plant analysis will appear here"
    )

    with gr.Row():
        gr.Markdown(f"""
<div class="legend-card" style="margin-top:14px;padding:14px;border:1px solid var(--border-color-primary)
;border-radius:10px;">




  <h4 style="margin-bottom:10px; font-size:20px; font-weight:600;">
🌿 Plant Status
</h4>


  <span style="color:{STATUS_OK_COLOR};font-size:26px;">●</span>
  <b>Healthy</b> – All sensor values within normal ranges<br>

  <span style="color:{STATUS_WARN_COLOR};font-size:26px;">●</span>
  <b>Warning</b> – At least one value near threshold<br>

  <span style="color:{STATUS_BAD_COLOR};font-size:26px;">●</span>
  <b>Not OK</b> – One or more values out of range<br><br>

  <span>Status is calculated automatically from sensor data</span>
</div>
        """)

        gr.Markdown(f"""
<div class="legend-card" style="margin-top:14px;padding:14px;border:1px solid var(--border-color-primary)
;border-radius:10px;">




  <h4 style="margin-bottom:10px; font-size:20px; font-weight:600;">
ℹ️ Valid Value Ranges
</h4>


  <span style="color:{COLOR_TEMP};font-size:26px;">●</span>
  🌡️ <b>Temperature</b>: 18–32°C<br>

  <span style="color:{COLOR_HUM};font-size:26px;">●</span>
  💧 <b>Air Humidity</b>: 35–75%<br>

  <span style="color:{COLOR_SOIL};font-size:26px;">●</span>
  🌱 <b>Soil Moisture</b>: 20–60%<br><br>

  <span>⚠️ Values outside these ranges are considered abnormal</span>
</div>
        """)

    gr.Markdown("""
<h2 style="text-align:center; margin-top:22px; font-size:26px; font-weight:600;">
📈 Plant Sensor Graphs
</h2>
""")

    with gr.Row():
        plot_temp = gr.Plot(label="Temperature")
        plot_hum = gr.Plot(label="Air Humidity")

    with gr.Row():
        plot_soil = gr.Plot(label="Soil Moisture")
        plot_combined = gr.Plot(label="Combined (Normalized)")

    overall_btn.click(
        fn=plant_dashboard,
        inputs=[samples],
        outputs=[overall_status, overall_info, plot_temp, plot_hum, plot_soil, plot_combined]
    )

✅ TAB 3 LOGIC - 📄 Generate Report

In [65]:
# ---------- TAB: Generate Report (LOGIC) ----------

def unify_sensor_dfs(dfs: dict) -> pd.DataFrame:
    """
    Robustly convert unicorn dfs into a single dataframe:
    supports timestamp as column OR as index.
    Output columns:
    timestamp | temperature | humidity | soil
    """
    def prep(df, col):
        if df is None or df.empty:
            return pd.DataFrame(columns=["timestamp", col])

        out = df.copy()

        # Case 1: timestamp is an index
        if "timestamp" not in out.columns:
            if out.index.name is not None:
                out = out.reset_index()
            else:
                # unnamed index → assume it's timestamp
                out = out.reset_index().rename(columns={"index": "timestamp"})

        # Now timestamp should exist
        if "timestamp" not in out.columns or "value" not in out.columns:
            return pd.DataFrame(columns=["timestamp", col])

        out = out[["timestamp", "value"]]
        out["timestamp"] = pd.to_datetime(out["timestamp"], errors="coerce")
        out = out.dropna(subset=["timestamp"])
        out = out.rename(columns={"value": col})
        return out

    t = prep(dfs.get("temperature"), "temperature")
    h = prep(dfs.get("humidity"), "humidity")
    s = prep(dfs.get("soil"), "soil")

    df = t.merge(h, on="timestamp", how="outer").merge(s, on="timestamp", how="outer")
    df = df.sort_values("timestamp").reset_index(drop=True)
    return df



def create_docx_report(dfs: dict, limit: int) -> str:
    """
    Creates a DOCX report in a combined-like style (English), without AI:
    - Executive Summary (rules-based)
    - Environmental Conditions table (Current / Average / Range)
    - Statistical Summary
    """
    df = unify_sensor_dfs(dfs)

    if df is None or df.empty:
        raise ValueError("No data available for report")

    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.dropna(subset=["timestamp"])

    # choose "daily" as last 100 rows (like combined did)
    daily = df.tail(100)

    # helper stats safely
    def col_stats(col):
        if col not in daily.columns:
            return None
        series = daily[col].dropna()
        if series.empty:
            return None
        return {
            "current": float(series.iloc[-1]),
            "avg": float(series.mean()),
            "min": float(series.min()),
            "max": float(series.max()),
        }

    stats_temp = col_stats("temperature")
    stats_hum = col_stats("humidity")
    stats_soil = col_stats("soil")

    # -------- Rules-based Executive Summary (English) --------
    def build_summary():
        parts = []
        parts.append(f"Daily plant health report generated from the latest sensor readings (up to {limit} samples per sensor).")

        issues = []
        # simple thresholds (adjust if you want)
        if stats_soil and stats_soil["avg"] < 30:
            issues.append("Soil moisture is low on average, which may cause water stress.")
        if stats_hum and stats_hum["avg"] < 40:
            issues.append("Humidity is low, which can increase dryness and stress for some plants.")
        if stats_hum and stats_hum["avg"] > 70:
            issues.append("Humidity is high, which may increase fungal risk without proper ventilation.")
        if stats_temp and stats_temp["avg"] > 30:
            issues.append("Temperature is relatively high on average; heat stress is possible.")
        if stats_temp and stats_temp["avg"] < 15:
            issues.append("Temperature is relatively low on average; cold stress is possible.")

        if not issues:
            parts.append("Overall environmental conditions look stable and within common recommended ranges.")
        else:
            parts.append("Potential risks identified based on averages:")
            parts.extend([f"- {x}" for x in issues])

        recs = []
        if stats_soil and stats_soil["avg"] < 30:
            recs.append("Increase watering and verify soil drainage.")
        if stats_hum and stats_hum["avg"] < 40:
            recs.append("Increase humidity (misting/humidifier) and avoid dry airflow.")
        if stats_hum and stats_hum["avg"] > 70:
            recs.append("Improve ventilation to reduce fungal risk.")
        if stats_temp and stats_temp["avg"] > 30:
            recs.append("Move the plant to a cooler/shaded area and monitor watering.")
        if stats_temp and stats_temp["avg"] < 15:
            recs.append("Move the plant to a warmer spot and avoid cold drafts.")

        # keep it short like combined (3-4 paragraphs)
        if not recs:
            parts.append("Recommendations: keep monitoring trends and maintain the current care routine.")
        else:
            parts.append("Recommendations:")
            parts.extend([f"- {r}" for r in recs[:3]])

        return "\n".join(parts)

    summary_text = build_summary()

    # -------- Build DOCX --------
    doc = Document()

    title = doc.add_heading("🌱 Daily Plant Health Report", 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER

    date_para = doc.add_paragraph()
    date_run = date_para.add_run(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    date_run.bold = True
    date_para.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Executive Summary
    doc.add_heading("Executive Summary", 1)
    doc.add_paragraph(summary_text)

    # Environmental Conditions Table
    doc.add_heading("Environmental Conditions", 1)

    table = doc.add_table(rows=4, cols=4)
    table.style = "Light Grid Accent 1"

    headers = table.rows[0].cells
    headers[0].text = "Parameter"
    headers[1].text = "Current"
    headers[2].text = "Average"
    headers[3].text = "Range"

    # Temperature row
    row1 = table.rows[1].cells
    row1[0].text = "🌡️ Temperature"
    if stats_temp:
        row1[1].text = f"{stats_temp['current']:.1f}°C"
        row1[2].text = f"{stats_temp['avg']:.1f}°C"
        row1[3].text = f"{stats_temp['min']:.1f}-{stats_temp['max']:.1f}°C"
    else:
        row1[1].text = row1[2].text = row1[3].text = "—"

    # Humidity row
    row2 = table.rows[2].cells
    row2[0].text = "💧 Humidity"
    if stats_hum:
        row2[1].text = f"{stats_hum['current']:.1f}%"
        row2[2].text = f"{stats_hum['avg']:.1f}%"
        row2[3].text = f"{stats_hum['min']:.1f}-{stats_hum['max']:.1f}%"
    else:
        row2[1].text = row2[2].text = row2[3].text = "—"

    # Soil row
    row3 = table.rows[3].cells
    row3[0].text = "🌱 Soil Moisture"
    if stats_soil:
        row3[1].text = f"{stats_soil['current']:.1f}%"
        row3[2].text = f"{stats_soil['avg']:.1f}%"
        row3[3].text = f"{stats_soil['min']:.1f}-{stats_soil['max']:.1f}%"
    else:
        row3[1].text = row3[2].text = row3[3].text = "—"

    # Statistical Summary
    doc.add_heading("Statistical Summary", 1)

    start = df["timestamp"].min()
    end = df["timestamp"].max()

    stats_text = (
        f"Total Readings: {len(df)}\n"
        f"Time Period: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}\n"
        "Data Points: Temperature, Humidity, Soil Moisture\n"
        "Quality: Based on available API readings"
    )
    doc.add_paragraph(stats_text)

    fd, path = tempfile.mkstemp(suffix=".docx", prefix="daily_report_")
    os.close(fd)
    doc.save(path)
    return path



def generate_report_screen(limit: int):
    """
    Button handler for the Generate Report tab.
    Returns: (status_text, file_path_or_None)
    """
    try:
        dfs = {
            "temperature": load_iot_data("temperature", limit),
            "humidity": load_iot_data("humidity", limit),
            "soil": load_iot_data("soil", limit),
        }

        if all(df is None or df.empty for df in dfs.values()):
            return "No data available to generate a report.", None

        out_path = create_docx_report(dfs, limit)
        return "✅ Report generated successfully. Download below:", out_path

    except Exception as e:
        return f"❌ Error generating report: {str(e)}", None


✅ TAB 3 GUI - 📄 Generate Report

In [66]:
# ---------- TAB: Generate Report (GUI) ----------

def build_generate_report_tab():
    gr.Markdown("## 📄 Generate Report")
    gr.Markdown(
        "Generate a Word (DOCX) report based on sensor data: temperature, humidity, and soil moisture."
    )

    report_samples = gr.Slider(
        minimum=5,
        maximum=200,
        value=20,
        step=1,
        label="Number of samples per sensor"
    )

    report_btn = gr.Button("📥 Generate & Download Report", variant="primary")
    report_status = gr.Textbox(label="Status", lines=2)
    report_file = gr.File(label="Download DOCX")

    report_btn.click(
        fn=generate_report_screen,
        inputs=[report_samples],
        outputs=[report_status, report_file]
    )

TAB 4 LOGIC - 🖼️ Plant Disease Detection

In [67]:
def analyze_plant(image, temp, humidity, soil):
    preds = clf(image)
    top = preds[0]

    label = top["label"]
    score = top["score"]

    alerts = []
    advice = []

    # --- Conditions based on sensors / user input ---
    if soil < 25:
        alerts.append("Low soil moisture")
        advice.append("Recommendation: irrigate / water the plant")

    if humidity > 80:
        alerts.append("High humidity")
        advice.append("Recommendation: improve ventilation (reduce fungal risk)")

    if temp > 40:
        alerts.append("High temperature")
        advice.append("Recommendation: move the plant to a shaded area")

    # --- Color status ("flag") based on image prediction ---
    # Simple rule: if label contains "healthy" => good (green), else bad (red)
    is_bad = ("healthy" not in label.lower())

    status_html = (
        "<div style='padding:10px;border-radius:10px;"
        f"background:{'#ffdddd' if is_bad else '#ddffdd'};"
        f"border:1px solid {'#ff0000' if is_bad else '#00aa00'};"
        "font-weight:700;'>"
        f"{'🔴 Plant status: BAD' if is_bad else '🟢 Plant status: GOOD'}"
        "</div>"
    )

    if not alerts:
        alerts.append("Status looks normal")

    return (
        f"Detected disease: {label} ({score:.2%})",
        status_html,
        "\n".join(alerts),
        "\n".join(advice)
    )

TAB 4 GUI - 🖼️ Plant Disease Detection

In [68]:
def build_plant_disease_detection_tab():
    gr.Markdown("## 🖼️ Plant Disease Detection")

    with gr.Row():

        # -------- LEFT SIDE --------
        with gr.Column(scale=2):

            image = gr.Image(
                type="filepath",
                label="Upload plant image",
                sources=["upload"]
            )

            temp = gr.Slider(0, 45, value=25, label="Temperature (°C)")
            humidity = gr.Slider(0, 100, value=50, label="Humidity (%)")
            soil = gr.Slider(0, 100, value=50, label="Soil Moisture (%)")

            run_btn = gr.Button("Analyze Plant", variant="primary")

        # -------- RIGHT SIDE --------
        with gr.Column(scale=2):

            diagnosis = gr.Textbox(
                label="Diagnosis",
                placeholder="Plant disease diagnosis will appear here"
            )

            status = gr.HTML(label="Status")

            alerts = gr.Textbox(
                label="Alerts",
                lines=5,
                placeholder="Sensor alerts and warnings will appear here"
            )

            recommendations = gr.Textbox(
                label="Recommendations",
                lines=5,
                placeholder="Care and treatment recommendations will appear here"
            )

    run_btn.click(
        fn=analyze_plant,
        inputs=[image, temp, humidity, soil],
        outputs=[diagnosis, status, alerts, recommendations]
    )

In [69]:
def build_placeholder_tab(title: str, note: str = "כאן ייכנס הקוד בהמשך"):
    gr.Markdown(f"## {title}")
    gr.Markdown(note)


# -----------------------------
# TABS (EMPTY PLACEHOLDERS)
# -----------------------------

def build_iot_dashboard_tab():
    build_placeholder_tab("📊 IoT Dashboard")


def build_search_engine_tab():
    build_placeholder_tab("🔍 Search Engine")

def build_rag_chat_tab():
    build_placeholder_tab("💬 RAG Chat")

def build_sync_data_tab():
    build_placeholder_tab("🔄 Sync Data")




In [70]:
# ✅ רשימת הטאבים — זה המקום היחיד שמוסיפים/מורידים טאבים
TABS = [
    ("🌱 Realtime Dashboard", build_realtime_dashboard_tab),
    ("📊 IoT Dashboard", build_iot_dashboard_tab),
    ("📄 Generate Report", build_generate_report_tab),
    ("🖼️ Plant Disease Detection", build_plant_disease_detection_tab),
    ("🔍 Search Engine", build_search_engine_tab),
    ("💬 RAG Chat", build_rag_chat_tab),
    ("🔄 Sync Data", build_sync_data_tab),
]

✅ APP BUILDER

In [71]:
def build_app() -> gr.Blocks:
    with gr.Blocks(title=APP_TITLE, css=CUSTOM_CSS) as demo:
        # Header
        gr.Markdown(
            f"<h1 style='text-align:left;font-size:30px;font-weight:700;margin:0;'>{APP_TITLE}</h1>"
        )
        gr.Markdown(
            f"<h1 style='text-align:left;font-size:20px;font-weight:700;margin:0;'>{APP_SUBTITLE}</h1>"
        )


        # Tabs
        with gr.Tabs():
            for tab_name, tab_builder in TABS:
                with gr.Tab(tab_name):
                    tab_builder()

    return demo

✅ LAUNCH (ONLY ONE)

In [72]:
if __name__ == "__main__":
    app = build_app()
    app.launch()

/tmp/ipython-input-29142094.py:2: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title=APP_TITLE, css=CUSTOM_CSS) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c400ebb4df4b43c0d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
